<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_evaluation_deepeval_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q langchain-groq deepeval asyncio pandas langchain langchain_google_genai vertexai instructor pydantic google-generativeai langchain_mistralai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.8/460.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━

#Loading data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir -p data
!find /content/drive/My\ Drive/tcc/resultados/ -type f ! -name '*.gsheet' -exec cp {} data/ \;

In [4]:
import os
import glob

def get_latest_file(directory, pattern):
    file_list = glob.glob(os.path.join(directory, pattern))

    if not file_list:
        raise FileNotFoundError("No files found matching the pattern.")

    latest_file = max(file_list, key=os.path.getmtime)

    return latest_file

    # return open(latest_file, 'r')



In [20]:
directory = 'data/'
pattern = 'ADVANCED*.csv'


latest_advanced_file = get_latest_file(directory, pattern)
print(latest_advanced_file)

data/ADVANCED_27_10_2024_14-46-00.csv


# Loading variables

In [6]:
from google.colab import userdata

class Settings:
  GROQ_API_KEY = userdata.get('GROQ_API_KEY')
  GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
  MISTRAL_AI_KEY = userdata.get('MISTRAL_AI_KEY')
  GROQ_API_KEY2 = userdata.get('GROQ_API_KEY2')
  GLHF_API_KEY  = userdata.get('GLHF_API_KEY')
  VERTEX_ENDPOINT = userdata.get('VERTEX_ENDPOINT')
  MISTRAL_AI_KEY2 = userdata.get('MISTRAL_AI_KEY2')



settings = Settings()

# Configuring LLMs for evaluation

In [7]:
import json
import random
from typing import get_origin, get_args, List, Dict, Union
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM
from pydantic import BaseModel, ValidationError
from time import sleep

class CustomGroqModel(DeepEvalBaseLLM):
    def __init__(self, model_id="llama-3.2-90b-text-preview"):
        self.MAX_RETRIES = 3
        self.api_keys = [settings.GROQ_API_KEY, settings.GROQ_API_KEY2]
        self.model_id = model_id

    def load_model(self, api_key):
        return ChatGroq(
            model=self.model_id,
            temperature=0.0,
            max_retries=4,
            api_key=api_key
        )

    def get_default_value(self, field_type):
        # Handle Union types
        if get_origin(field_type) is Union:
            args = get_args(field_type)
            # Try to find a non-None type in the Union
            for arg in args:
                if arg is not type(None):
                    return self.get_default_value(arg)
            return None

        # Handle List types
        if get_origin(field_type) is list:
            return []

        # Handle Dict types
        if get_origin(field_type) is dict:
            return {}

        # Handle basic types
        type_defaults = {
            str: "",
            int: 0,
            float: 0.0,
            bool: False,
            dict: {},
            list: [],
        }

        # If it's a BaseModel, recursively generate null schema
        if isinstance(field_type, type) and issubclass(field_type, BaseModel):
            return self.generate_null_schema(field_type)

        return type_defaults.get(field_type, None)

    def generate_null_schema(self, schema: BaseModel) -> BaseModel:
        try:
            default_values = {}
            for field_name, field in schema.__fields__.items():
                field_type = field.outer_type_
                default_values[field_name] = self.get_default_value(field_type)

            return schema.parse_obj(default_values)
        except ValidationError as e:
            print(f"Schema validation error: {e}")
            return None

    def generate(self, prompt: str, schema: BaseModel, attempt=0) -> BaseModel:
        selected_api_key = random.choice(self.api_keys)
        chat_model = self.load_model(selected_api_key)

        if schema:
            structured_llm = chat_model.with_structured_output(schema)
        else:
            print(prompt)
            return chat_model.invoke(prompt).content

        try:
            response = structured_llm.invoke(prompt)
            print(response)
            return response
        except Exception as e:
            print(f"Error generating response: {str(e)}, Attempt {attempt + 1}")

            if attempt < self.MAX_RETRIES:
                sleep(1)  # Add a small delay before retrying
                return self.generate(prompt, schema, attempt + 1)
            else:
                print(f"Max retries exceeded. Returning null schema.")
                return self.generate_null_schema(schema)

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:

        return self.generate(prompt, schema)

    def get_model_name(self):
        return self.model_id

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from deepeval.models.base_model import DeepEvalBaseLLM

class GoogleGenerativeAI(DeepEvalBaseLLM):
    """Class to implement Google Generative AI for DeepEval"""

    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()

        response = chat_model.invoke(prompt).content
        print(response)
        return response

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Google Generative AI Model"

gemini_llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-latest",
    convert_system_message_to_human=True,
    google_api_key=userdata.get('GEMINI_API_KEY'),
    temperature=0.3,
    top_k=1,
    top_p=0.9,
    max_output_tokens=8192,
    verbose=True
)

gemini = GoogleGenerativeAI(model=gemini_llm)

In [9]:
from pydantic import BaseModel
import google.generativeai as genai
import instructor

class CustomGeminiFlash(DeepEvalBaseLLM):
    def __init__(self):
        genai.configure(api_key = settings.GEMINI_API_KEY)
        self.model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        client = self.load_model()
        instructor_client = instructor.from_gemini(
            client=client,
            mode=instructor.Mode.GEMINI_JSON,
        )
        resp = instructor_client.messages.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            response_model=schema,
        )
        print(resp)
        return resp

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Gemini 1.5 Flash"

In [10]:
import json
import random
from typing import get_origin, get_args, List, Dict, Union
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI
from langchain_openai import ChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
from pydantic import BaseModel, ValidationError
from time import sleep

class LangchainModel(DeepEvalBaseLLM):
    def __init__(self, provider="Groq", model_id="llama-3.2-90b-text-preview"):
        self.MAX_RETRIES = 3
        self.model_id = model_id
        self.provider = provider
        self.request_count = 0  # Adicionado contador de requests
        self.api_keys = (
            [settings.GROQ_API_KEY, settings.GROQ_API_KEY2] if provider == "Groq"
            else [settings.MISTRAL_AI_KEY] if provider == "Mistral"
            else [settings.GLHF_API_KEY]
        )
        self.model = self.load_model()

    def load_model(self):
        if self.provider == "Groq":
            selected_api_key = random.choice(self.api_keys)
            return ChatGroq(model=self.model_id, temperature=0.0, max_retries=4, api_key=selected_api_key)
        elif self.provider == "Mistral":
            return ChatMistralAI(model=self.model_id, temperature=0, max_retries=4, api_key=settings.MISTRAL_AI_KEY)
        elif self.provider == "VertexAI":
            # Atribuindo o endpoint baseado nas configurações e utilizando Vertex AI
            endpoint = settings.VERTEX_ENDPOINT
            # return ChatOpenAI(
            #     api_key=credentials.token,
            #     base_url=endpoint,
            #     model=self.model_id,
            #     temperature=0,
            #     max_retries=4
            # )

    def get_default_value(self, field_type):
        if get_origin(field_type) is Union:
            for arg in get_args(field_type):
                if arg is not type(None):
                    return self.get_default_value(arg)
            return None

        if get_origin(field_type) is list:
            return []

        if get_origin(field_type) is dict:
            return {}

        type_defaults = {str: "", int: 0, float: 0.0, bool: False, dict: {}, list: []}

        if isinstance(field_type, type) and issubclass(field_type, BaseModel):
            return self.generate_null_schema(field_type)

        return type_defaults.get(field_type, None)

    def generate_null_schema(self, schema: BaseModel) -> BaseModel:
        try:
            default_values = {field_name: self.get_default_value(field.outer_type_) for field_name, field in schema.__fields__.items()}
            return schema.parse_obj(default_values)
        except ValidationError as e:
            print(f"Schema validation error: {e}")
            return None

    def generate(self, prompt: str, schema: BaseModel = None, attempt=0) -> BaseModel:
        chat_model = self.load_model()

        # Incrementa o contador de requests
        self.request_count += 1
        print(f"Request number: {self.request_count}")


        if schema:
            structured_llm = chat_model.with_structured_output(schema)
        else:
            return chat_model.invoke(prompt).content

        try:
            response = structured_llm.invoke(prompt)
            return response
        except Exception as e:
            # print(str(e))
            if attempt < self.MAX_RETRIES:
                sleep(10)
                return self.generate(prompt, schema, attempt + 1)
            return self.generate_null_schema(schema)

    async def a_generate(self, prompt: str, schema: BaseModel = None) -> BaseModel:
        try:
            chat_model = self.load_model()
            if schema:
                structured_llm = chat_model.with_structured_output(schema)
                return await structured_llm.ainvoke(prompt)
            else:
                return await chat_model.ainvoke(prompt)
        except Exception as e:
            return self.generate(prompt, schema)


    def get_model_name(self):
        return self.model_id


# Loading test cases

In [21]:
from deepeval.test_case import LLMTestCase
import pandas as pd

def context_handler(contexts):
    context_list = [context for context in contexts.split('\n<|FIM_DO_CHUNK|>\n') if context]
    if len(context_list) > 4:
        # Calculate the number of elements to concatenate into 4 strings
        chunk_size = len(context_list) // 4 + (1 if len(context_list) % 4 != 0 else 0)
        return [' '.join(context_list[i:i + chunk_size]) for i in range(0, len(context_list), chunk_size)][:4]
    return context_list

def load_test_cases(file_path):
    df = pd.read_csv(file_path)
    test_cases = [
        LLMTestCase(
            input=row['Pergunta'],
            actual_output=row['RespostaGerada'],
            expected_output=row['RespostaIdeal'],
            retrieval_context=context_handler(row['Contexto']),
            additional_metadata = {
                'PromptTokens': row['PromptTokens'],
                'CompletionTokens': row['CompletionTokens'],
                'TotalTokens': row['TotalTokens'],
                'ConfiguracaoDeInferencia': row['ConfiguracaoDeInferencia'],
                'Documento': row['Documento']
            }
        )
        for _, row in df.iterrows()
    ]

    return test_cases


In [22]:
latest_advanced_file

'data/ADVANCED_27_10_2024_14-46-00.csv'

In [23]:
df = pd.read_csv(latest_advanced_file)
df.sample()

,Documento,Pergunta,RespostaIdeal,Diretriz_de_Avaliacao,RespostaGerada,PromptTokens,TotalTokens,CompletionTokens,Contexto,ConsultaExpandida,ConfiguracaoDeInferencia,Metodo
9,Ambos,Programação I saiu da grade?,"Sim, a disciplina Programação I saiu da grade ...",A resposta ideal deve conter os seguintes pont...,### Resposta Final\n\n**PPC 2018:**\n- A disci...,36741,38550,1809,"PPC 2018 Context:\nNão, a disciplina de Progra...","Programação I saiu da grade?, [""Programação I ...","ADVANCED: Expansão: llama-3.1-70b, RAG: open-...",advanced


# Configuring deepeval metrics

In [24]:
from deepeval.metrics import AnswerRelevancyMetric, ContextualRecallMetric, ContextualPrecisionMetric, FaithfulnessMetric, GEval, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCaseParams

def generate_deepeval_metrics(model, model_for_context):
    # Define correctness metric with model set
    correctness_metric = GEval(
        name="Correctness",
        criteria="Determine whether the actual output is factually correct based on the expected output.",
        evaluation_steps=[
            "Check whether the facts in 'actual output' contradict any facts in 'expected output'",
            "You should also heavily penalize omission of detail",
            "Vague language, or contradicting OPINIONS, are OK"
        ],
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
        model=model
    )

    answer_relevancy_metric = AnswerRelevancyMetric(
        model=model,
        threshold=0.5
    )

    contextual_recall_metric = ContextualRecallMetric(
        threshold=0.25,
        model=model_for_context,
        include_reason = False
    )

    contextual_precision_metric = ContextualPrecisionMetric(
        threshold=0.5,
        model=model,
    )

    contextual_relevancy = ContextualRelevancyMetric(
        model = model,
        threshold=0.5
    )

    faithfulness_metric = FaithfulnessMetric(
        model=model,
        threshold=0.5,
        truths_extraction_limit = 3
    )

    # Return the metrics in a list
    return [
        correctness_metric,
        answer_relevancy_metric,
        contextual_recall_metric,
        # contextual_relevancy,
        faithfulness_metric
    ]


In [25]:
import json
import os


def parse_test_result_to_dict(test_result):
    parsed_results = []

    for input_key, test_result_list in test_result.items():
        for test_result_obj in test_result_list:
            result_dict = {
                'input': input_key,
                'success': test_result_obj.success,
                'conversational': test_result_obj.conversational,
                'multimodal': test_result_obj.multimodal,
                'actual_output': test_result_obj.actual_output,
                'expected_output': test_result_obj.expected_output,
                'context': test_result_obj.context,
                'retrieval_context': test_result_obj.retrieval_context
            }

            metrics_list = []
            for metric in test_result_obj.metrics_data:
                metric_dict = {
                    'name': metric.name,
                    'threshold': metric.threshold,
                    'success': metric.success,
                    'score': metric.score,
                    'reason': metric.reason,
                    'strict_mode': metric.strict_mode,
                    'evaluation_model': metric.evaluation_model,
                    'error': metric.error,
                    'evaluation_cost': metric.evaluation_cost,
                    'verbose_logs': metric.verbose_logs
                }
                metrics_list.append(metric_dict)

            result_dict['metrics'] = metrics_list
            parsed_results.append(result_dict)

    return parsed_results


def save_dict_to_json_on_drive(data_dict, drive_folder_path, drive_file_name):
    os.makedirs(drive_folder_path, exist_ok=True)
    drive_path = f'{drive_folder_path}/{os.path.basename(drive_file_name)}'
    with open(drive_path, 'w', encoding='utf-8') as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False)
    print(f'File saved to: {drive_path}')

    return drive_path

import time
import hashlib

def transform_and_save_results(results, latest_file,  identifier='Groq_'):
    # Cria um hash curto a partir do timestamp
    timestamp = str(int(time.time()))
    hash_suffix = hashlib.md5(timestamp.encode()).hexdigest()[:6]  # Gera um hash curto com 6 caracteres

    # Cria o nome do arquivo com o identificador e o hash curto
    filename = identifier + hash_suffix + '_' + latest_file.split('/')[1].replace('.csv', '.json')

    # Salva os resultados no arquivo JSON
    return save_dict_to_json_on_drive(results, f'/content/drive/MyDrive/tcc/resultados/validados/deepeval/{identifier}', filename)



# Dataset evaluation

In [26]:
from deepeval import evaluate
from deepeval.dataset import EvaluationDataset
from typing import List, Dict, Any, Optional, Tuple
from time import sleep

def create_metric_result(measurement) -> Dict:
    metrics_data = measurement.metrics_data[0]

    return {
        'name': metrics_data.name,
        'threshold': metrics_data.threshold,
        'success': metrics_data.success,
        'score': metrics_data.score,
        'reason': metrics_data.reason,
        'strict_mode': metrics_data.strict_mode,
        'evaluation_model': metrics_data.evaluation_model,
        'error': metrics_data.error,
        'evaluation_cost': metrics_data.evaluation_cost,
        'verbose_logs': metrics_data.verbose_logs
    }


def evaluate_metric_with_retry(test_case, metric, sleep_times: Optional[Tuple[int, int]], retries_left: int) -> Dict:
    try:
        dataset = EvaluationDataset(test_cases=[test_case])
        measurement = dataset.evaluate([metric])
        test_result = measurement.test_results[0]
        metric_result = create_metric_result(test_result)

        return metric_result
    except Exception as e:
        print(f"Error evaluating metric: {str(e)}")

        if retries_left > 0:
            print(f"Retrying... Attempts left: {retries_left}")
            sleep(10)
            return evaluate_metric_with_retry(test_case, metric, sleep_times, retries_left - 1)
        else:
            print(f"Failed to evaluate metric after all retry attempts.")
            # Return a metric result with invalid values and an error field
            return {
                'name': type(metric).__name__,
                'threshold': None,
                'success': True,
                'score': -1,  # Example of an invalid score value
                'reason': 'Evaluation failed',
                'strict_mode': None,
                'evaluation_model': None,
                'error': str(e),
                'evaluation_cost': 0,  # No cost since it failed
                'verbose_logs': 'No logs available due to failure.'
            }


def evaluate_test_case(test_case, metrics, sleep_times: Optional[Tuple[int, int]] = None, max_retries: int = 1) -> Dict:
    metric_results = []
    overall_success = True

    for index, metric in enumerate(metrics):
        metric_result = evaluate_metric_with_retry(test_case, metric, sleep_times, max_retries)

        if not metric_result['success']:
            overall_success = False

        metric_results.append(metric_result)

        if sleep_times:
            sleep(sleep_times[0])  # Sleep after each metric evaluation

    return {
        'input': test_case.input,
        'success': overall_success,
        'actual_output': test_case.actual_output,
        'expected_output': test_case.expected_output,
        'context': test_case.context,
        'retrieval_context': test_case.retrieval_context,
        'additional_metadata': test_case.additional_metadata,
        'metrics': metric_results
    }

def evaluate_model(model, model_for_context, test_cases: List, sleep_times: Optional[Tuple[int, int]] = None) -> List[Dict]:

    model_metrics = generate_deepeval_metrics(model, model_for_context)
    results = []

    for test_case in test_cases:
        try:
            evaluation_result = evaluate_test_case(test_case, model_metrics, sleep_times)
            results.append(evaluation_result)

            if sleep_times:
                sleep(sleep_times[1])  # Sleep after each test case evaluation
        except Exception as e:
            print(f"Failed to evaluate test case: {str(e)}")
            continue

    return results


In [27]:
def divide_test_cases(test_cases, num_parts):
    part_size = len(test_cases) // num_parts
    remainder = len(test_cases) % num_parts

    parts = []
    start_index = 0

    for i in range(num_parts):
        # Adicionar 1 caso extra às primeiras 'remainder' partes para balancear
        end_index = start_index + part_size + (1 if i < remainder else 0)
        parts.append(test_cases[start_index:end_index])
        start_index = end_index

    return parts

def load_results_from_file(file_path):
    with open(file_path, 'r') as file:
        results = json.load(file)
    return results


def combine_and_save_files(output_files, final_output_name):
    combined_results = []

    for file in output_files:
        results = load_results_from_file(file)
        combined_results.extend(results)

    transform_and_save_results(combined_results, final_output_name, 'mistral-large-latest_')


def evaluation_pipeline(model, model_for_context, sleep_times, file_pattern):
    latest_file = get_latest_file('data/', file_pattern)
    print(latest_file)
    test_cases = load_test_cases(latest_file)


    if file_pattern.startswith("ADVANCED"):
        parts = divide_test_cases(test_cases, 10)
        output_files = []
        # Avaliar cada parte e salvar resultados
        for i, part in enumerate(parts):
            if i < 9:
                continue
            print(f"Avaliando parte {i + 1} de {len(parts)}")
            results = evaluate_model(model, model_for_context, part, sleep_times)
            output_file = transform_and_save_results(results, latest_file, f'mistral-large-latest_unit_')
            output_files.append(output_file)

        return output_files
    else:
        # Avaliação normal para outros padrões de arquivo
        results = evaluate_model(model, test_cases, sleep_times)
        transform_and_save_results(results, latest_file, 'mistral-large-latest' + '_')


In [28]:
mixtral = LangchainModel('Mistral', 'mistral-large-latest')
llama = LangchainModel('Groq', 'llama-3.1-70b-versatile')

In [ ]:
# evaluation_pipeline(mixtral, (1,1), 'NAIVE*')

In [ ]:
list_of_files = evaluation_pipeline(mixtral, mixtral, (1,1), 'ADVANCED*')

data/ADVANCED_19_10_2024_23-29-34.csv
Avaliando parte 10 de 10


✨ You're running DeepEval's latest Correctness (GEval) Metric! (using mistral-large-latest, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:02,  2.61s/test case]



Metrics Summary

  - ✅ Correctness (GEval) (score: 1.0, threshold: 0.5, strict: False, evaluation model: mistral-large-latest, reason: The actual output provides detailed information about the possibility of taking EAD courses, including specific regulations and percentages, which aligns with the input question., error: None)

For test case:

  - input: Há possibilidade de realizar disciplinas EAD?
  - actual output: Sim, há possibilidade de realizar disciplinas EAD. De acordo com o PPC, a Portaria do MEC N.º2.117, de 6 de dezembro de 2019, prevê a oferta de carga horária na modalidade de EaD dos cursos de graduação presenciais, até o limite de 40% da carga horária total do curso. Além disso, a Resolução N.º5/CONSUNI- CGRAD/UFFS/2014 define a possibilidade de ofertar até 20% da carga horária do curso em atividades semipresenciais, incluindo-se nesse percentual tanto os componentes curriculares integralmente a distância quanto a fração da carga horária ministrada a distância nos compo

...

3


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:15, 15.32s/test case]



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: mistral-large-latest, reason: The score is 1.00 because there are no contradictions! Great job!, error: None)

For test case:

  - input: O curso oferece vagas para transferência externa ou interna?
  - actual output: Sim, o curso oferece vagas para transferência externa e interna.

A transferência interna ocorre mediante a troca de turno, de curso ou de campus no âmbito da UFFS, sendo vedada a transferência interna no semestre de ingresso ou de retorno para a UFFS.

A transferência externa é a concessão de vaga a discente regularmente matriculado em outra instituição de ensino superior, nacional ou estrangeira, para prosseguimento de seus estudos na UFFS.

O acesso aos cursos de graduação da UFFS, tanto no que diz respeito ao preenchimento das vagas de oferta regular, como das ofertas de caráter especial e das eventuais vagas ociosas, se dá por meio de diferentes formas de ingresso, inc

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

File saved to: /content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_f85d1f_ADVANCED_19_10_2024_23-29-34.json


In [ ]:
list_of_files

['/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_f85d1f_ADVANCED_19_10_2024_23-29-34.json']

In [ ]:
old_files_no_prefix = ['mistral-large-latest_unit_6135f2_ADVANCED_19_10_2024_23-29-34.json', 'mistral-large-latest_unit_fbfbef_ADVANCED_19_10_2024_23-29-34.json', 'mistral-large-latest_unit_b672a3_ADVANCED_19_10_2024_23-29-34.json',
             'mistral-large-latest_unit_308f1e_ADVANCED_19_10_2024_23-29-34.json', 'mistral-large-latest_unit_ceae35_ADVANCED_19_10_2024_23-29-34.json', 'mistral-large-latest_unit_7e8159_ADVANCED_19_10_2024_23-29-34.json',
             'mistral-large-latest_unit_2b4bba_ADVANCED_19_10_2024_23-29-34.json', 'mistral-large-latest_unit_184839_ADVANCED_19_10_2024_23-29-34.json', 'mistral-large-latest_unit_1750e9_ADVANCED_19_10_2024_23-29-34.json'
             ]
old_files = [f'/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/{file}' for file in old_files_no_prefix]
old_files

['/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_6135f2_ADVANCED_19_10_2024_23-29-34.json',
 '/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_fbfbef_ADVANCED_19_10_2024_23-29-34.json',
 '/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_b672a3_ADVANCED_19_10_2024_23-29-34.json',
 '/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_308f1e_ADVANCED_19_10_2024_23-29-34.json',
 '/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_ceae35_ADVANCED_19_10_2024_23-29-34.json',
 '/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_7e8159_ADVANCED_19_10_2024_23-29-34.json',
 '/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-lar

In [ ]:
all_files = list_of_files + old_files

In [ ]:
combine_and_save_files(all_files ,get_latest_file('data/', 'ADVANCED*'))

File saved to: /content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_/mistral-large-latest_fd4aeb_ADVANCED_19_10_2024_23-29-34.json


In [ ]:
import pandas as pd

d = pd.read_json('/content/drive/MyDrive/tcc/resultados/validados/deepeval/mistral-large-latest_unit_/mistral-large-latest_unit_24b4d1_ADVANCED_19_10_2024_23-29-34.json')
d.head()

,input,success,actual_output,expected_output,context,retrieval_context,additional_metadata,metrics
0,Qual é a ementa de Cálculo Numérico?,True,A ementa de Cálculo Numérico é:\n\nErros compu...,Erros computacionais e aproximação numérica. C...,NaN,[\n\n| Código | COMPONENTE CURRICULAR | Crédit...,"{'PromptTokens': 3596, 'CompletionTokens': 114...","[{'name': 'Correctness (GEval)', 'threshold': ..."
1,Qual é a ementa de Sistemas Digitais?,True,"A ementa de Sistemas Digitais, conforme o PPC,...",Dispositivos Lógicos Programáveis. Introdução ...,NaN,[\n\n| Código | COMPONENTE CURRICULAR | Crédit...,"{'PromptTokens': 3322, 'CompletionTokens': 159...","[{'name': 'Correctness (GEval)', 'threshold': ..."


In [ ]:
teste = load_test_cases(get_latest_file('data/', 'ADVANCED*'))
teste[22]

LLMTestCase(input='Qual é a ementa de Grafos?', actual_output='A ementa de Grafos, conforme o PPC, é:\n\n"Definições básicas de Teoria dos Grafos: vizinhança, isomorfismo, subgrafo, clique e con- junto independente. Representação computacional de grafos. Modelagem de problemas com grafos. Grafos bipartidos e cortes. Caminhos, ciclos, diâmetro e cintura. Conexidade e al- cançabilidade. Grafos biconexos e articulações. Grafos dirigidos e caminho mínimo. Busca. Árvores e árvores geradoras de custo mínimo. Grafos eulerianos e hamiltonianos. Estabili- dade e emparelhamento. Fluxo. Coloração e Planaridade. Problemas da Cobertura e da Tra- vessia."', expected_output='Definições básicas de Teoria dos Grafos: vizinhança, isomorfismo, subgrafo, clique e conjunto independente. Representação computacional de grafos. Modelagem de problemas com grafos. Grafos bipartidos e cortes. Caminhos, ciclos, diâmetro e cintura. Conexidade e alcançabilidade. Grafos biconexos e articulações. Grafos dirigidos e c

In [ ]:
teste[67]

LLMTestCase(input='Qual é a ementa de Grafos?', actual_output='A ementa de Grafos, conforme o PPC, é:\n\nDefinições básicas de Teoria dos Grafos. Representação computacional de grafos. Modelagem de problemas com grafos. Caminhos e ciclos. Conexidade e alcançabilidade. Grafos dirigidos e caminhos de custo mínimo. Busca. Árvores e árvores geradoras de custo mínimo. Grafos eulerianos e hamiltonianos. Estabilidade e emparelhamento. Fluxo. Coloração e Planaridade.', expected_output='Definições básicas de Teoria dos Grafos. Representação computacional de grafos. Modelagem de problemas com grafos. Caminhos e ciclos. Conexidade e alcançabilidade. Grafos dirigidos e caminhos de custo mínimo. Busca. Árvores e árvores geradoras de custo mínimo. Grafos eulerianos e hamiltonianos. Estabilidade e emparelhamento. Fluxo. Coloração e Planaridade.', context=None, retrieval_context=['\n\nConceitos básicos. Dispositivos gráficos. Sistemas de cores. Transformações geométricas. \nPrimitivas gráficas. Visibi

In [ ]:
testes = [teste[22], teste[67]]
metrics = generate_deepeval_metrics(mixtral)

dataset = EvaluationDataset(test_cases=testes)
measurement = dataset.evaluate([metrics[2]])

✨ You're running DeepEval's latest Contextual Recall Metric! (using llama-3.1-70b-versatile, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:10,  5.17s/test case]



Metrics Summary

  - ✅ Contextual Recall (score: 1.0, threshold: 0.25, strict: False, evaluation model: llama-3.1-70b-versatile, reason: None, error: None)

For test case:

  - input: Qual é a ementa de Grafos?
  - actual output: A ementa de Grafos, conforme o PPC, é:

"Definições básicas de Teoria dos Grafos: vizinhança, isomorfismo, subgrafo, clique e con- junto independente. Representação computacional de grafos. Modelagem de problemas com grafos. Grafos bipartidos e cortes. Caminhos, ciclos, diâmetro e cintura. Conexidade e al- cançabilidade. Grafos biconexos e articulações. Grafos dirigidos e caminho mínimo. Busca. Árvores e árvores geradoras de custo mínimo. Grafos eulerianos e hamiltonianos. Estabili- dade e emparelhamento. Fluxo. Coloração e Planaridade. Problemas da Cobertura e da Tra- vessia."
  - expected output: Definições básicas de Teoria dos Grafos: vizinhança, isomorfismo, subgrafo, clique e conjunto independente. Representação computacional de grafos. Modelagem de pr

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [ ]:
isinstance(type(metrics[2]).__name__, str)


True

In [ ]:
# llama_32_90b = LangchainModel('Groq', 'llama-3.2-90b-text-preview')
# llama_results = evaluate_model(llama_32_90b, test_cases, (5, 30))
# transform_and_save_results(llama_results, 'llama-3.2-90b-text-preview_')

In [ ]:
# llama_3_70b = LangchainModel('Groq', 'llama-3.1-70b-versatile')
# llama_results = evaluate_model(llama_3_70b, test_cases, (1, 10))
# transform_and_save_results(llama_results, 'llama-3.1-70b-versatile_')

In [ ]:
# ChatOpenAI(
#     api_key=credentials.token,
#     base_url=settings.VERTEX_ENDPOINT,
#     model="meta/llama3-405b-instruct-maas",
#     temperature=0,
#     max_retries=4
# ).invoke('Oi Llama, posso te utilizar para avaliar sistemas RAG?')

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field


# class Joke(BaseModel):
#     setup: str = Field(description="The setup of the joke")
#     punchline: str = Field(description="The punchline to the joke")

# llama_3_405b = LangchainModel("VertexAI", "meta/llama3-405b-instruct-maas")

# print(llama_3_405b.generate('A good joke, please', None))

In [ ]:
# llama_3_405b = LangchainModel("VertexAI", "meta/llama3-405b-instruct-maas")
# metrics = generate_deepeval_metrics(llama_3_405b)

# metrics[0].measure(test_cases[0])

In [ ]:
# llama_3_405b = LangchainModel("VertexAI", "meta/llama3-405b-instruct-maas")
# # llama- = generate_deepeval_metrics(llama_3_405b)
# llama_3_405b_results = evaluate_model(llama_3_405b, test_cases[0:1], False)
# # transform_and_save_results(llama_3_405b_results, 'LLAMA-3.1-705B_')

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using meta/llama3-405b-instruct-maas, 
strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]

Request number: 1


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:03,  3.68s/test case]

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]

Request number: 2
Request number: 3
Error code: 400 - [{'error': {'code': 400, 'message': 'Request contains an invalid argument.', 'status': 'INVALID_ARGUMENT'}}]
Request number: 4
Error code: 400 - [{'error': {'code': 400, 'message': 'Request contains an invalid argument.', 'status': 'INVALID_ARGUMENT'}}]
Request number: 5
Error code: 400 - [{'error': {'code': 400, 'message': 'Request contains an invalid argument.', 'status': 'INVALID_ARGUMENT'}}]
Request number: 6


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:19, ?test case/s]

Error code: 400 - [{'error': {'code': 400, 'message': 'Request contains an invalid argument.', 'status': 'INVALID_ARGUMENT'}}]
Failed to evaluate test case: 'AnswerRelevancyMetric' object has no attribute 'name'
